In [1]:
import findspark 
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.appName('i_s3_postgres').getOrCreate()
spark

In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
import os
import configparser

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:2.7.6" pyspark-shell'

config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))

aws_profile = 'default'
access_key = config.get(aws_profile, "aws_access_key_id").replace("'", "")
secret_key = config.get(aws_profile, "aws_secret_access_key").replace("'", "")

sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)


In [4]:
%%time
bucket_name = "attemp-glue"
key_file = "gestion.pe/2018/12/*.csv"
path_csv_spark = 's3n://{}/{}'.format(bucket_name, key_file)


opts = {'header': True, 'sep': ',', 'inferSchema':False}
df_csv = spark.read.csv(
    path_csv_spark, **opts
)
df_csv.show()

+----------+-------------+---+
|       dia|page_avg_time|mes|
+----------+-------------+---+
|2018-12-18|           61| 12|
|2018-12-18|           44| 12|
|2018-12-18|            0| 12|
|2018-12-18|           67| 12|
|2018-12-18|           15| 12|
|2018-12-18|           39| 12|
|2018-12-18|            0| 12|
|2018-12-18|           24| 12|
|2018-12-18|            0| 12|
|2018-12-18|           40| 12|
|2018-12-18|           15| 12|
|2018-12-18|           79| 12|
|2018-12-18|          131| 12|
|2018-12-18|            0| 12|
|2018-12-18|           20| 12|
|2018-12-18|           19| 12|
|2018-12-18|            5| 12|
|2018-12-18|           10| 12|
|2018-12-18|           26| 12|
|2018-12-18|           29| 12|
+----------+-------------+---+
only showing top 20 rows

CPU times: user 14.3 ms, sys: 2.18 ms, total: 16.5 ms
Wall time: 40.4 s


In [5]:
df_csv.columns

['dia', 'page_avg_time', 'mes']

In [6]:
print(dir(df_csv.write.partitionBy()))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_df', '_jwrite', '_set_opts', '_spark', '_sq', 'bucketBy', 'csv', 'format', 'insertInto', 'jdbc', 'json', 'mode', 'option', 'options', 'orc', 'parquet', 'partitionBy', 'save', 'saveAsTable', 'sortBy', 'text']


In [16]:
bucket_output = "athena-attemp"
path_output_spark = 's3a://{}/'.format(bucket_output)
print(path_output_spark)

"""
df_csv.\
    write.\
    partitionBy("dia").\
    mode('overwrite').\
    save(path=path_output_spark, format='parquet')
"""

df_csv.write.partitionBy("dia").parquet(path_output_spark, mode="overwrite")

s3a://athena-attemp/
